In [19]:
import os
import itertools
from pathlib import Path
import numpy as np
import pandas as pd
import 
from PIL import Image


## Loading in image data

First we need to load in all the image data from the `./data` folder. These images are in TIFF format that will be used to create a coordinate system and able to

This will be implemented as a `generator` in order to save memory.


In [5]:
# def load_tiff_images() -> np.ndarray:

img_path = Path("./data/cell_image_2.tiff")
if not img_path.exists():
    raise FileNotFoundError(f"Unable to find: {str(img_path)}")
if img_path.suffix == "tiff":
    raise TypeError(f"Image must be .tiff format not {img_path.suffix}")

tiff_img = Image.open(img_path)
tiff_arr = np.array(tiff_img)


In [22]:
tiff_img.size


(1600, 902)

In [23]:
def crop_image(fpath: str| Path, crop_width: int, crop_height: int ) -> List[]:
    """Crops images based on the given crop dimensions. Images are saved within fo

    Parameters
    ----------
    fpath : str | Path
        path to image
    crop_width : int
        crop width
    crop_height : int
        crop height
    
    Returns
    -------
    List[]
        Generates a directory containing all the cropped images. The images will
        have the source name along with image number id.
    """
    # type checking
    accepted_types = (str, Path)
    if not isinstance(fpath, accepted_types):
        _type = type(fpath)
        raise TypeError(f"'fpath' must str or path type not {_type}")
    # change to Path type if string
    if isinstance(fpath, str):
        fpath = Path(fpath)
    # checking crop width inputs
    if not isinstance(crop_height, int):
        raise TypeError(f"'crop_height' must integer not {type(crop_height)}")
    if not isinstance(crop_width, int):
        raise TypeError(f"'crop_width' must integer not {type(crop_height)}")

    # creating saving path
    source_name = fpath.stem

    # setting up crop dimensions
    crop_w, crop_h = (64, )
    savedir= Path("test_split")
    savedir.mkdir(exist_ok=True)


    # setting up cropping dimensions 
    x, y = (0, 0)
    width, height = tiff_img.size
    width_range_crop = range(0, width, crop_width)
    height_range_crop = range(0, height, crop_height)

    # selecting all images 
    # remove the ones that do not have the desired 
    
    # -- this will store image information indicating which images were selected
    # -- based on 
    # NOTE: make sure to download crop size, source, and id
    list_of_cropped_images = []
    for frame_num, (col_i, row_i) in enumerate(itertools.product(width_range_crop, height_range_crop), start=1):

        crop = tiff_img.crop((col_i, row_i, col_i + crop_width, row_i + crop_height))
        save_to= os.path.join(str(savedir), "counter_{:03}.tiff")
        crop.save(save_to.format(frame_num))

SyntaxError: expected ':' (2090744851.py, line 1)

In [17]:
img = Image.open("./test_split/counter_001.tiff")
img.show()


In [21]:
Path("./test_split/counter_001.tiff").stem


'counter_001'

In [7]:
# Selecting random crop
# setting up coordinates
x_max, y_max = tiff_img.size

# bounding box
#  (left, upper, right, lower)
box = (0, 0, 500, 500)
size = 400

# selected a random point
x_rand = np.random.randint(0, x_max)
y_rand = np.random.randint(0, y_max)

print(x_rand, y_rand)


NameError: name 'tiff_img' is not defined

## Create a coordinate plane on images

once the images are loaded
